In [1]:
import pyspark
from pyspark.sql import SparkSession
import os, sys

spark = SparkSession.builder.master('local[*]').getOrCreate()

In [2]:
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['HADOOP_HOME'] = "C:\\apps\hadoop"
os.environ['SPARK_HOME'] = "C:\\apps\spark-3.5.0-bin-hadoop3"

# Spark SQL

In [3]:
df = spark.sql('''select "OK" as Status''')
df.show()

+------+
|Status|
+------+
|    OK|
+------+



# Import data

In [4]:
df = spark.read.csv('./cereal.csv', sep=',', inferSchema=True, header=True)

print(f'Lines: {df.count()}')
df.show(10)

Lines: 77
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple

In [5]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



# Manipulate date with Spark SQL

In [6]:
df.createOrReplaceTempView('cereal_sql')

In [7]:
df_cereal = spark.sql('''SELECT * FROM cereal_sql''')
df_cereal.show(5)

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
+---------------

In [8]:
filter = spark.sql('''SELECT * FROM cereal_sql WHERE type = "C"''')
print(f'Lines: {filter.count()}')
filter.show(5)

Lines: 74
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
+-----

# Counting lines as new column

In [9]:
filter = spark.sql('''SELECT COUNT(*) AS total FROM cereal_sql WHERE mfr = "K"''')
filter.show()

+-----+
|total|
+-----+
|   23|
+-----+



# Variações do comando Select

In [10]:
# Colunas específicas
cereal = spark.sql('''SELECT name,type,mfr FROM cereal_sql ''')
cereal.show(5)

+--------------------+----+---+
|                name|type|mfr|
+--------------------+----+---+
|           100% Bran|   C|  N|
|   100% Natural Bran|   C|  Q|
|            All-Bran|   C|  K|
|All-Bran with Ext...|   C|  K|
|      Almond Delight|   C|  R|
+--------------------+----+---+
only showing top 5 rows



In [11]:
# Valores distintos
cereal = spark.sql('''SELECT type,mfr FROM cereal_sql ''')
print(f'Lines: {cereal.count()}')
cereal.show(5)

cereal = spark.sql('''SELECT DISTINCT type,mfr FROM cereal_sql ''')
print(f'Lines: {cereal.count()}')
cereal.show(5)

Lines: 77
+----+---+
|type|mfr|
+----+---+
|   C|  N|
|   C|  Q|
|   C|  K|
|   C|  K|
|   C|  R|
+----+---+
only showing top 5 rows

Lines: 9
+----+---+
|type|mfr|
+----+---+
|   C|  P|
|   C|  Q|
|   C|  N|
|   H|  Q|
|   C|  R|
+----+---+
only showing top 5 rows



# Comando Where

In [12]:
cereal = spark.sql('''SELECT DISTINCT type,mfr FROM cereal_sql WHERE type = "C"''')
print(f'Lines: {cereal.count()}')
cereal.show(5)

Lines: 6
+----+---+
|type|mfr|
+----+---+
|   C|  P|
|   C|  Q|
|   C|  N|
|   C|  R|
|   C|  G|
+----+---+
only showing top 5 rows



In [13]:
# Dupla condição
cereal = spark.sql('''SELECT * FROM cereal_sql WHERE mfr = "K" AND calories >= 100''')
print(f'Lines: {cereal.count()}')
cereal.show(5)

Lines: 19
+------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|              name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|       Apple Jacks|  K|   C|     110|      2|  0|   125|  1.0| 11.0|    14|    30|      25|    2|   1.0| 1.0|33.174094|
|       Corn Flakes|  K|   C|     100|      2|  0|   290|  1.0| 21.0|     2|    35|      25|    1|   1.0| 1.0|45.863324|
|         Corn Pops|  K|   C|     110|      1|  0|    90|  1.0| 13.0|    12|    20|      25|    2|   1.0| 1.0|35.782791|
|Cracklin' Oat Bran|  K|   C|     110|      3|  3|   140|  4.0| 10.0|     7|   160|      25|    3|   1.0| 0.5|40.448772|
|           Crispix|  K|   C|     110|      2|  0|   220|  1.0| 21.0|     3|    30|      25|    3|   1.0| 1.0|46.895644|
+------------------+--

# Groupby

In [14]:
# Agrupamento simples
cereal = spark.sql(''' SELECT mfr,type,COUNT(*) AS count FROM cereal_sql GROUP BY mfr,type''')
cereal.show()

+---+----+-----+
|mfr|type|count|
+---+----+-----+
|  A|   H|    1|
|  P|   C|    9|
|  K|   C|   23|
|  G|   C|   22|
|  Q|   C|    7|
|  R|   C|    8|
|  Q|   H|    1|
|  N|   H|    1|
|  N|   C|    5|
+---+----+-----+



In [15]:
# Agrupamento com mais de uma agregação
cereal = spark.sql(''' SELECT mfr,type,COUNT(*) AS count, SUM(calories) as total_calories FROM cereal_sql GROUP BY mfr,type''')
cereal.show()

+---+----+-----+--------------+
|mfr|type|count|total_calories|
+---+----+-----+--------------+
|  A|   H|    1|           100|
|  P|   C|    9|           980|
|  K|   C|   23|          2500|
|  G|   C|   22|          2450|
|  Q|   C|    7|           660|
|  R|   C|    8|           920|
|  Q|   H|    1|           100|
|  N|   H|    1|           100|
|  N|   C|    5|           420|
+---+----+-----+--------------+



In [16]:
# Agrupamento com mais de uma agregação
cereal = spark.sql(''' SELECT \
                           mfr, type, COUNT(*) AS count, SUM(calories) as total_calories \
                       FROM cereal_sql\
                       GROUP BY mfr,type''')
cereal.show()

+---+----+-----+--------------+
|mfr|type|count|total_calories|
+---+----+-----+--------------+
|  A|   H|    1|           100|
|  P|   C|    9|           980|
|  K|   C|   23|          2500|
|  G|   C|   22|          2450|
|  Q|   C|    7|           660|
|  R|   C|    8|           920|
|  Q|   H|    1|           100|
|  N|   H|    1|           100|
|  N|   C|    5|           420|
+---+----+-----+--------------+



# CASE WHEN

In [17]:
cereal = spark.sql(''' SELECT DISTINCT\
                           type \
                       FROM cereal_sql''')
cereal.show()

+----+
|type|
+----+
|   C|
|   H|
+----+



In [18]:
cereal = spark.sql(''' SELECT\
                           mfr, type, (case when type = "C" then "tipo C" else "tipo H" end) as category, count(*) as count \
                       FROM cereal_sql\
                       GROUP BY mfr, type''')
cereal.show()

+---+----+--------+-----+
|mfr|type|category|count|
+---+----+--------+-----+
|  A|   H|  tipo H|    1|
|  P|   C|  tipo C|    9|
|  K|   C|  tipo C|   23|
|  G|   C|  tipo C|   22|
|  Q|   C|  tipo C|    7|
|  R|   C|  tipo C|    8|
|  Q|   H|  tipo H|    1|
|  N|   H|  tipo H|    1|
|  N|   C|  tipo C|    5|
+---+----+--------+-----+



In [19]:
# Células comentadas em SQL
cereal = spark.sql(''' SELECT
                           mfr, type,
                           (case
                                when type = "C" then "tipo C"
                                --when type = "H" then "tipo B"
                                --else "tipo H"
                           end) as category,
                           count(*) as count 
                       FROM cereal_sql
                       GROUP BY mfr, type''')
cereal.show()

+---+----+--------+-----+
|mfr|type|category|count|
+---+----+--------+-----+
|  A|   H|    NULL|    1|
|  P|   C|  tipo C|    9|
|  K|   C|  tipo C|   23|
|  G|   C|  tipo C|   22|
|  Q|   C|  tipo C|    7|
|  R|   C|  tipo C|    8|
|  Q|   H|    NULL|    1|
|  N|   H|    NULL|    1|
|  N|   C|  tipo C|    5|
+---+----+--------+-----+



# Consultas avançadas

In [20]:
cereal = spark.sql('''  SELECT mfr, type,
                                sum(calories) as sum_calories,
                                min(calories) as min_calories,
                                max(calories) as max_calories,
                                avg(calories) as avg_calories,
                                count(name) as count
                        FROM cereal_sql
                        GROUP BY mfr,type
                        ORDER BY count DESC

''')

cereal.show()

+---+----+------------+------------+------------+------------------+-----+
|mfr|type|sum_calories|min_calories|max_calories|      avg_calories|count|
+---+----+------------+------------+------------+------------------+-----+
|  K|   C|        2500|          50|         160|108.69565217391305|   23|
|  G|   C|        2450|         100|         140|111.36363636363636|   22|
|  P|   C|         980|          90|         120|108.88888888888889|    9|
|  R|   C|         920|          90|         150|             115.0|    8|
|  Q|   C|         660|          50|         120| 94.28571428571429|    7|
|  N|   C|         420|          70|          90|              84.0|    5|
|  A|   H|         100|         100|         100|             100.0|    1|
|  Q|   H|         100|         100|         100|             100.0|    1|
|  N|   H|         100|         100|         100|             100.0|    1|
+---+----+------------+------------+------------+------------------+-----+



In [21]:
cereal = spark.sql('''  SELECT mfr, type,
                                sum(calories) as sum_calories,
                                min(calories) as min_calories,
                                max(calories) as max_calories,
                                avg(calories) as avg_calories,
                   
                                sum(carbo) as sum_carbo,
                                min(carbo) as min_carbo,
                                max(carbo) as max_carbo,
                                avg(carbo) as avg_carbo,
                   
                                sum(vitamins) as sum_vitamins,
                                min(vitamins) as min_vitamins,
                                max(vitamins) as max_vitamins,
                                avg(vitamins) as avg_vitamins,
                   
                                count(name) as count
                        FROM cereal_sql
                        GROUP BY mfr,type
                        ORDER BY count DESC

''')

cereal.show()

+---+----+------------+------------+------------+------------------+---------+---------+---------+------------------+------------+------------+------------+------------------+-----+
|mfr|type|sum_calories|min_calories|max_calories|      avg_calories|sum_carbo|min_carbo|max_carbo|         avg_carbo|sum_vitamins|min_vitamins|max_vitamins|      avg_vitamins|count|
+---+----+------------+------------+------------+------------------+---------+---------+---------+------------------+------------+------------+------------+------------------+-----+
|  K|   C|        2500|          50|         160|108.69565217391305|    348.0|      7.0|     22.0|15.130434782608695|         800|          25|         100| 34.78260869565217|   23|
|  G|   C|        2450|         100|         140|111.36363636363636|    324.0|     10.5|     21.0|14.727272727272727|         775|          25|         100| 35.22727272727273|   22|
|  P|   C|         980|          90|         120|108.88888888888889|    119.0|     11.0|  

# Formatação de colunas

In [22]:
# Formatação de valores decimais
cereal = spark.sql('''  SELECT mfr, type,
                                sum(calories) as sum_calories,
                                min(calories) as min_calories,
                                max(calories) as max_calories,
                                cast(avg(calories) as decimal(10,2)) as avg_calories,
                   
                                sum(carbo) as sum_carbo,
                                min(carbo) as min_carbo,
                                max(carbo) as max_carbo,
                                cast(avg(carbo) as decimal(10,2)) as avg_carbo,
                   
                                sum(vitamins) as sum_vitamins,
                                min(vitamins) as min_vitamins,
                                max(vitamins) as max_vitamins,
                                cast(avg(vitamins) as decimal(10,2)) as avg_vitamins,
                   
                                count(name) as count
                        FROM cereal_sql
                        GROUP BY mfr,type
                        ORDER BY count DESC

''')

cereal.show()

+---+----+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+-----+
|mfr|type|sum_calories|min_calories|max_calories|avg_calories|sum_carbo|min_carbo|max_carbo|avg_carbo|sum_vitamins|min_vitamins|max_vitamins|avg_vitamins|count|
+---+----+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+-----+
|  K|   C|        2500|          50|         160|      108.70|    348.0|      7.0|     22.0|    15.13|         800|          25|         100|       34.78|   23|
|  G|   C|        2450|         100|         140|      111.36|    324.0|     10.5|     21.0|    14.73|         775|          25|         100|       35.23|   22|
|  P|   C|         980|          90|         120|      108.89|    119.0|     11.0|     17.0|    13.22|         225|          25|          25|       25.00|    9|
|  R|   C|         920|          9

# Case When Avançado

In [23]:
# Formatação de valores decimais
cereal = spark.sql('''  SELECT mfr, type,
                                (case
                                    when mfr = "K" then "Abacaxi"
                                    when mfr = "G" then "Goiaba"
                                    when mfr = "P" then "Pera"
                                    when mfr = "R" then "Roma"
                                    when mfr = "Q" then "Espinafre"
                                    when mfr = "N" then "Nectarina"
                                    when mfr = "A" then "Abacate"
                                    else "NA"
                                end) as type_fruit,
                                sum(calories) as sum_calories,
                                min(calories) as min_calories,
                                max(calories) as max_calories,
                                cast(avg(calories) as decimal(10,2)) as avg_calories,
                   
                                sum(carbo) as sum_carbo,
                                min(carbo) as min_carbo,
                                max(carbo) as max_carbo,
                                cast(avg(carbo) as decimal(10,2)) as avg_carbo,
                   
                                sum(vitamins) as sum_vitamins,
                                min(vitamins) as min_vitamins,
                                max(vitamins) as max_vitamins,
                                cast(avg(vitamins) as decimal(10,2)) as avg_vitamins,
                   
                                count(name) as count
                        FROM cereal_sql
                        GROUP BY mfr,type
                        ORDER BY count DESC

''')

cereal.show()

+---+----+----------+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+-----+
|mfr|type|type_fruit|sum_calories|min_calories|max_calories|avg_calories|sum_carbo|min_carbo|max_carbo|avg_carbo|sum_vitamins|min_vitamins|max_vitamins|avg_vitamins|count|
+---+----+----------+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+-----+
|  K|   C|   Abacaxi|        2500|          50|         160|      108.70|    348.0|      7.0|     22.0|    15.13|         800|          25|         100|       34.78|   23|
|  G|   C|    Goiaba|        2450|         100|         140|      111.36|    324.0|     10.5|     21.0|    14.73|         775|          25|         100|       35.23|   22|
|  P|   C|      Pera|         980|          90|         120|      108.89|    119.0|     11.0|     17.0|    13.22|         225|          25| 

# Joins

## InnerJoin

In [24]:
df_sales = spark.read.csv('./sales_data_sample.csv', sep=',', inferSchema=True, header=True)

print(f'Lines: {df_sales.count()}')
df_sales.show(10)

Lines: 2823
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003 0:

In [25]:
df_sales.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

In [26]:
df_sales.createOrReplaceTempView('sales_sql')

In [27]:
df_sales.show()

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003

### Tabelas dimensionais

In [28]:
# Tabela de datas
calendar = spark.sql('''
                  SELECT DISTINCT ORDERDATE, QTR_ID, MONTH_ID, YEAR_ID
                  FROM sales_sql
                  ORDER BY ORDERDATE

''')
calendar.createOrReplaceTempView('calendar')

calendar.show(5)

+--------------+------+--------+-------+
|     ORDERDATE|QTR_ID|MONTH_ID|YEAR_ID|
+--------------+------+--------+-------+
|1/10/2003 0:00|     1|       1|   2003|
|1/10/2005 0:00|     1|       1|   2005|
|1/12/2004 0:00|     1|       1|   2004|
|1/12/2005 0:00|     1|       1|   2005|
|1/15/2004 0:00|     1|       1|   2004|
+--------------+------+--------+-------+
only showing top 5 rows



In [29]:
# Tabela de vendas/pedidos
orders = spark.sql('''
                  SELECT DISTINCT ORDERNUMBER, CUSTOMERNAME, QUANTITYORDERED, SALES, PRICEEACH, PRODUCTCODE, ORDERLINENUMBER
                  FROM sales_sql
                  ORDER BY ORDERNUMBER

''')

orders.createOrReplaceTempView('orders')

orders.show(5)

+-----------+--------------------+---------------+-------+---------+-----------+---------------+
|ORDERNUMBER|        CUSTOMERNAME|QUANTITYORDERED|  SALES|PRICEEACH|PRODUCTCODE|ORDERLINENUMBER|
+-----------+--------------------+---------------+-------+---------+-----------+---------------+
|      10100|Online Diecast Cr...|             22|1903.22|    86.51|   S18_4409|              4|
|      10100|Online Diecast Cr...|             50| 3390.0|     67.8|   S18_2248|              2|
|      10100|Online Diecast Cr...|             49|1689.03|    34.47|   S24_3969|              1|
|      10100|Online Diecast Cr...|             30| 5151.0|    100.0|   S18_1749|              3|
|      10101|Blauer See Auto, Co.|             26|3773.38|    100.0|   S18_2795|              1|
+-----------+--------------------+---------------+-------+---------+-----------+---------------+
only showing top 5 rows



In [30]:
# Tabela de clientes
customer = spark.sql('''
                  SELECT DISTINCT CUSTOMERNAME, PHONE, ADDRESSLINE1, ADDRESSLINE2, CITY, STATE, POSTALCODE, COUNTRY, TERRITORY
                  FROM sales_sql
                  ORDER BY CUSTOMERNAME

''')

customer.createOrReplaceTempView('customer')

customer.show(5)

+--------------------+--------------+--------------------+------------+------------+-----+----------+---------+---------+
|        CUSTOMERNAME|         PHONE|        ADDRESSLINE1|ADDRESSLINE2|        CITY|STATE|POSTALCODE|  COUNTRY|TERRITORY|
+--------------------+--------------+--------------------+------------+------------+-----+----------+---------+---------+
|      AV Stores, Co.|(171) 555-1555|   Fauntleroy Circus|        NULL|  Manchester| NULL|   EC2 5NT|       UK|     EMEA|
|        Alpha Cognac|    61.77.6555|1 rue Alsace-Lorr...|        NULL|    Toulouse| NULL|     31000|   France|     EMEA|
|  Amica Models & Co.|   011-4988555| Via Monte Bianco 34|        NULL|      Torino| NULL|     10100|    Italy|     EMEA|
|Anna's Decoration...|  02 9936 8555|   201 Miller Street|    Level 15|North Sydney|  NSW|      2060|Australia|     APAC|
|   Atelier graphique|    40.32.2555|      54, rue Royale|        NULL|      Nantes| NULL|     44000|   France|     EMEA|
+--------------------+--

In [31]:
print(calendar.count())
print(orders.count())
print(customer.count())

252
2823
92


### Tabela master

In [32]:
master = spark.sql('''
                SELECT *
                FROM orders o
                INNER JOIN customer c on o.CUSTOMERNAME = c.CUSTOMERNAME                 
''')

print(master.count())
master.show(5)

2823
+-----------+--------------------+---------------+-------+---------+-----------+---------------+--------------------+----------------+--------------------+------------+------------+-----+----------+-------+---------+
|ORDERNUMBER|        CUSTOMERNAME|QUANTITYORDERED|  SALES|PRICEEACH|PRODUCTCODE|ORDERLINENUMBER|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|        CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|
+-----------+--------------------+---------------+-------+---------+-----------+---------------+--------------------+----------------+--------------------+------------+------------+-----+----------+-------+---------+
|      10308|       Mini Classics|             20| 4570.4|    100.0|   S10_4698|              1|       Mini Classics|      9145554562|3758 North Pendal...|        NULL|White Plains|   NY|     24067|    USA|       NA|
|      10272|Diecast Classics ...|             35| 5818.4|    100.0|   S12_1108|              2|Diecast Classics ...|      2155

In [33]:
master = spark.sql('''
                SELECT DISTINCT o.ORDERNUMBER, C.CITY
                FROM orders o
                INNER JOIN customer c on o.CUSTOMERNAME = c.CUSTOMERNAME                 
''')

print(master.count())
master.show(5)

307
+-----------+----------+
|ORDERNUMBER|      CITY|
+-----------+----------+
|      10300| Frankfurt|
|      10385|San Rafael|
|      10182|San Rafael|
|      10140|Burlingame|
|      10241|Strasbourg|
+-----------+----------+
only showing top 5 rows

